**Importing required packages**

In [1]:
import sqlite3
import pandas as pd

**Setting up connection to database**

In [2]:
conn = sqlite3.connect('Db-IMDB.db')

**Test Query**

In [3]:
pd.read_sql_query('SELECT * FROM Movie LIMIT 10', conn)

,index,MID,title,year,rating,num_votes
0,0,tt2388771,Mowgli,2018,6.6,21967
1,1,tt5164214,Ocean's Eight,2018,6.2,110861
2,2,tt1365519,Tomb Raider,2018,6.4,142585
3,3,tt0848228,The Avengers,2012,8.1,1137529
4,4,tt8239946,Tumbbad,2018,8.5,7483
5,5,tt7027278,Kedarnath,2018,5.5,1970
6,6,tt3498820,Captain America: Civil War,2016,7.8,536641
7,7,tt8108198,Andhadhun,2018,9.0,18160
8,8,tt3741834,Lion,2016,8.1,170216
9,9,tt6747420,Rajma Chawal,2018,5.7,681


### Question 1:

After Checking some queries I got to see some years having roman numbers in front of them

In [4]:
query = """
SELECT DISTINCT year
FROM Movie
WHERE CAST(year AS INT)=0
LIMIT 10
"""
pd.read_sql_query(query, conn)

,year
0,I 2009
1,I 2018
2,XVII 2016
3,I 2017
4,II 2018
5,I 2002
6,III 2016
7,I 2015
8,I 2016
9,I 2006


To Correct them I took last chars and turned them to integer

In [5]:
# Took code from src: https://stackoverflow.com/questions/12504985/how-to-take-last-four-characters-from-a-varchar
query = """
SELECT DISTINCT year, CAST(SUBSTR(year, LENGTH(year)-3, 4) AS INT) real_year
FROM Movie
WHERE CAST(year AS INT)=0
LIMIT 10
"""
pd.read_sql_query(query, conn)

,year,real_year
0,I 2009,2009
1,I 2018,2018
2,XVII 2016,2016
3,I 2017,2017
4,II 2018,2018
5,I 2002,2002
6,III 2016,2016
7,I 2015,2015
8,I 2016,2016
9,I 2006,2006


In [6]:
year_formula = 'CAST(SUBSTR(year, LENGTH(year)-3, 4) AS INT)'

From now I use above formula for year when its needed in this notebook

In [7]:
query = f"""
SELECT DISTINCT {year_formula} real_year
FROM Movie
WHERE CAST(real_year AS INT)=0
"""
pd.read_sql(query, conn)

,real_year


In above statement, I used string formatting to insert year formula and aliased it as real_year. As you can see above there are no years which give wrong result now.

Now checking movie names and years for 'Comedy' genre and having a leap year

In [8]:
# Checking Leap year code is taken from StackOverFlow
# src: https://stackoverflow.com/questions/6534788/check-for-leap-year
query = f"""
SELECT m.title, {year_formula} real_year
FROM Movie m JOIN M_Genre mg ON m.MID = mg.MID
WHERE ((real_year%4 = 0 AND real_year%100 <> 0) OR real_year%400 = 0)
AND mg.GID IN (
SELECT GID FROM Genre
WHERE Name LIKE '%Comedy%'
)
"""
pd.read_sql_query(query, conn)

,title,real_year
0,Mastizaade,2016
1,Harold & Kumar Go to White Castle,2004
2,Gangs of Wasseypur,2012
3,Around the World in 80 Days,2004
4,The Accidental Husband,2008
5,Barfi!,2012
6,Bride & Prejudice,2004
7,Beavis and Butt-Head Do America,1996
8,Dostana,2008
9,Kapoor & Sons,2016


We got 232 rows of movies which released in leap year and are comedy. Now building query for our **FINAL ANSWER**.

In [9]:
query = f"""
SELECT DISTINCT TRIM(p.Name) director_name, req_movies.title movie_title, req_movies.real_year year
FROM Person p JOIN M_Director md ON p.PID = md.PID
  JOIN
  (
    SELECT m.MID mid, m.title title, {year_formula} real_year FROM Movie m
    JOIN M_Genre mg ON m.MID = mg.MID
    WHERE ((real_year%4 = 0 AND real_year%100 <> 0) OR real_year%400 = 0)
    AND mg.GID IN (
      SELECT GID FROM Genre
      WHERE Name LIKE '%Comedy%'
    )
  ) req_movies ON md.MID = req_movies.mid
"""
pd.read_sql_query(query, conn)

,director_name,movie_title,year
0,Milap Zaveri,Mastizaade,2016
1,Danny Leiner,Harold & Kumar Go to White Castle,2004
2,Anurag Kashyap,Gangs of Wasseypur,2012
3,Frank Coraci,Around the World in 80 Days,2004
4,Griffin Dunne,The Accidental Husband,2008
5,Anurag Basu,Barfi!,2012
6,Gurinder Chadha,Bride & Prejudice,2004
7,Mike Judge,Beavis and Butt-Head Do America,1996
8,Tarun Mansukhani,Dostana,2008
9,Shakun Batra,Kapoor & Sons,2016


Got 232 Rows. Used Trim in above query as they are duplicate rows in Person table.

### Question 2:

Checking if there are several movies named 'Anand'

In [10]:
query = """
SELECT * FROM Movie WHERE title='Anand'
"""
pd.read_sql_query(query, conn)

,index,MID,title,year,rating,num_votes
0,66,tt0066763,Anand,1971,8.8,21616


As there is only one movie we can build the **FINAL QUERY**

In [11]:
query = """
SELECT Name FROM Person
WHERE PID IN (
SELECT TRIM(mc.PID)
FROM M_Cast mc JOIN Movie m ON m.MID = mc.MID
WHERE m.title='Anand'
)
"""
pd.read_sql_query(query, conn)

,Name
0,Amitabh Bachchan
1,Rajesh Khanna
2,Sumita Sanyal
3,Ramesh Deo
4,Seema Deo
5,Asit Kumar Sen
6,Dev Kishan
7,Atam Prakash
8,Lalita Kumari
9,Savita


Got 17 Actors for movie 'Anand'

### Question 3:

**FINAL QUERY**

In [12]:
query = f"""
SELECT DISTINCT TRIM(Name) Actor FROM Person
WHERE PID IN (
  SELECT TRIM(PID) FROM M_Cast
  WHERE MID IN (
    SELECT TRIM(MID) FROM Movie
    WHERE {year_formula} < 1970
  )
)

INTERSECT

SELECT DISTINCT TRIM(Name) Actor FROM Person
WHERE PID IN (
  SELECT TRIM(PID) FROM M_Cast
  WHERE MID IN (
    SELECT TRIM(MID) FROM Movie
    WHERE {year_formula} > 1990
  )
)
"""
pd.read_sql_query(query, conn)

,Actor
0,A.K. Hangal
1,Aachi Manorama
2,Abbas
3,Abdul
4,Abhi Bhattacharya
5,Achala Sachdev
6,Adil
7,Ajay
8,Ajit
9,Akashdeep


Got 424 Actors who acted before 1970 and after 1990.

### Question 4:

**FINAL QUERY**

In [13]:
query = """
SELECT DISTINCT TRIM(p.Name) director, temp.movies no_of_movies
FROM Person p JOIN (
  SELECT PID, COUNT(MID) movies FROM M_Director
  GROUP BY PID
  HAVING movies>=10
) temp ON p.PID = temp.PID
ORDER BY no_of_movies DESC
"""
pd.read_sql_query(query, conn)

,director,no_of_movies
0,David Dhawan,39
1,Mahesh Bhatt,35
2,Ram Gopal Varma,30
3,Priyadarshan,30
4,Vikram Bhatt,29
5,Hrishikesh Mukherjee,27
6,Yash Chopra,21
7,Shakti Samanta,19
8,Basu Chatterjee,19
9,Subhash Ghai,18


Got 58 directors who directed 10 movies or more.

### Question 5a:

I Find that there are only 42 movies in 1990 where in question it is shown to be 13522. I Assume that the data is changed (reduced) because total number of movie Id's in our database is only 3475.

In [14]:
query = f"""
SELECT {year_formula} real_year, COUNT(MID) FROM Movie
WHERE real_year=1990
GROUP BY real_year
"""
pd.read_sql_query(query, conn)

,real_year,COUNT(MID)
0,1990,42


In [15]:
query = """
SELECT COUNT(MID) FROM Movie
"""
pd.read_sql_query(query, conn)

,COUNT(MID)
0,3475


Below query gets count of female cast for each movie

In [16]:
query = """
SELECT MID, COUNT(TRIM(PID)) female_cast FROM M_Cast
WHERE TRIM(PID) IN (
SELECT TRIM(PID) FROM Person
WHERE TRIM(Gender)='Female'
)
GROUP BY MID
"""
pd.read_sql_query(query, conn)

,MID,female_cast
0,tt0021594,3
1,tt0026274,11
2,tt0027256,5
3,tt0028217,3
4,tt0031580,17
5,tt0033616,7
6,tt0036077,3
7,tt0038491,3
8,tt0039654,6
9,tt0040067,5


Now we calculate movies with only female cast by comparing total number of cast with above table

In [17]:
query = """
SELECT TRIM(MID), COUNT(TRIM(PID)) cast_count FROM M_Cast
WHERE TRIM(PID) IN (
  SELECT TRIM(PID) FROM Person
  WHERE Gender='Female'
)
GROUP BY MID

INTERSECT

SELECT TRIM(MID), COUNT(TRIM(PID)) cast_count FROM M_Cast
GROUP BY MID
"""
pd.read_sql_query(query, conn)

,TRIM(MID),cast_count
0,tt0272001,11
1,tt0354922,10
2,tt0375882,1
3,tt8458202,2


We get only 4 movies which have all female cast out of 3475 movies.

Now we build **FINAL QUERY**

In [18]:
query = f"""
SELECT {year_formula} real_year, COUNT(MID) no_of_female_cast_movies FROM Movie
WHERE MID IN (
SELECT MID FROM (
  SELECT MID, COUNT(TRIM(PID)) cast_count FROM M_Cast
  WHERE TRIM(PID) IN (
    SELECT TRIM(PID) FROM Person
    WHERE Gender='Female'
  )
  GROUP BY MID
  INTERSECT
  SELECT MID, COUNT(TRIM(PID)) cast_count FROM M_Cast
  GROUP BY MID
)
)
GROUP BY real_year
"""
pd.read_sql_query(query, conn)

,real_year,no_of_female_cast_movies
0,1939,1
1,1999,1
2,2000,1
3,2018,1


I assume the data is not completely given in db file provided (db file size 7332 KB) so we get very sparse result. And we got only 4 rows and for other years the result is 0.

### Question 5b:

We can biuld **FINAL QUERY** as we did most of the work in previous question.

In [19]:
query = f"""
SELECT table1.real_year, CAST(COALESCE(no_of_female_cast_movies, 0) AS FLOAT)*100/total_movies ratio, total_movies
FROM

(
  SELECT {year_formula} real_year, COUNT(MID) total_movies FROM Movie
  GROUP BY real_year
) table1

LEFT JOIN

(
  SELECT {year_formula} real_year, COUNT(MID) no_of_female_cast_movies FROM Movie
  WHERE MID IN (
    SELECT MID FROM (
      SELECT MID, COUNT(TRIM(PID)) cast_count FROM M_Cast
      WHERE TRIM(PID) IN (
        SELECT TRIM(PID) FROM Person
        WHERE Gender='Female'
      )
      GROUP BY MID
      
      INTERSECT
      
      SELECT MID, COUNT(TRIM(PID)) cast_count FROM M_Cast
      GROUP BY MID
    )
  )
  GROUP BY real_year
) table2

ON table1.real_year = table2.real_year
"""
pd.read_sql_query(query, conn)

,real_year,ratio,total_movies
0,1931,0.000000,1
1,1936,0.000000,3
2,1939,50.000000,2
3,1941,0.000000,1
4,1943,0.000000,1
5,1946,0.000000,2
6,1947,0.000000,2
7,1948,0.000000,3
8,1949,0.000000,3
9,1950,0.000000,2


We get 78 rows which has female_cast_movies ratio and total_movies count for all years

### Question 6:

Seeing if more than one movie has the maximum cast value.

In [20]:
query = """
SELECT MID, COUNT(DISTINCT TRIM(PID)) cast_count FROM M_Cast
GROUP BY MID
ORDER BY cast_count DESC
LIMIT 10
"""
pd.read_sql_query(query, conn)

,MID,cast_count
0,tt5164214,238
1,tt0451631,233
2,tt6173990,215
3,tt1188996,213
4,tt3498820,191
5,tt1981128,170
6,tt1573482,165
7,tt1190080,154
8,tt2120120,144
9,tt2510874,140


We get movie id with maximum cast from below query

In [21]:
query = """
SELECT MID, MAX(cast_count) FROM (
  SELECT MID, COUNT(DISTINCT TRIM(PID)) cast_count FROM M_Cast
  GROUP BY MID
)
"""
pd.read_sql_query(query, conn)

,MID,MAX(cast_count)
0,tt5164214,238


**FINAL QUERY**

In [22]:
query = """
SELECT title, cast_count
FROM Movie m JOIN (
  SELECT MID, MAX(cast_count) cast_count FROM (
    SELECT MID, COUNT(DISTINCT TRIM(PID)) cast_count FROM M_Cast
    GROUP BY MID
  )
) id ON m.MID = id.MID
"""
pd.read_sql_query(query, conn)

,title,cast_count
0,Ocean's Eight,238


"Ocean's Eight" is the movie with maximum cast of 238 distint people

### Question 7:

Let us write query for getting all decades that are in our database.

In [23]:
query = f"""
SELECT DISTINCT {year_formula} start_year, {year_formula}+9 end_year
FROM Movie
ORDER BY start_year
"""
pd.read_sql_query(query, conn)

,start_year,end_year
0,1931,1940
1,1936,1945
2,1939,1948
3,1941,1950
4,1943,1952
5,1946,1955
6,1947,1956
7,1948,1957
8,1949,1958
9,1950,1959


Now join this decades with Movies Table on condition that movies year is in between the start_year and end_year. This will give number of movies in each decade.

In [24]:
query = f"""
SELECT decades.start_year, decades.end_year, COUNT(DISTINCT m.MID) no_of_movies
FROM (
  SELECT MID, {year_formula} real_year FROM Movie
) m JOIN (
  SELECT DISTINCT {year_formula} start_year, {year_formula}+9 end_year
  FROM Movie
  ORDER BY start_year
) decades ON m.real_year BETWEEN decades.start_year AND decades.end_year
GROUP BY decades.start_year
ORDER BY no_of_movies DESC
"""
pd.read_sql_query(query, conn)

,start_year,end_year,no_of_movies
0,2008,2017,1205
1,2009,2018,1202
2,2007,2016,1188
3,2005,2014,1170
4,2006,2015,1160
5,2004,2013,1147
6,2003,2012,1114
7,2010,2019,1092
8,2002,2011,1090
9,2001,2010,1047


Now we get the Maximum value from the above table. Building **FINAL QUERY** to get the answer. Ans notice that we dont have two decades with maximum number of movies from above result as the answer is sorted according to no_of_movies.

In [25]:
query = f"""
SELECT start_year, end_year, MAX(no_of_movies) FROM (
  SELECT decades.start_year, decades.end_year, COUNT(DISTINCT m.MID) no_of_movies
  FROM (
    SELECT MID, {year_formula} real_year FROM Movie
  ) m JOIN (
    SELECT DISTINCT {year_formula} start_year, {year_formula}+9 end_year
    FROM Movie
    ORDER BY start_year
  ) decades ON m.real_year BETWEEN decades.start_year AND decades.end_year
  GROUP BY decades.start_year
)
"""
pd.read_sql_query(query, conn)

,start_year,end_year,MAX(no_of_movies)
0,2008,2017,1205


So The Decade [2008, 2017] has the maximum number of movies with 1205 movies made in between them.

### Question 8:

*src: https://stackoverflow.com/questions/6299950/sql-difference-between-rows*

Below code provides row number for each actor so actors having row number as only 1 did only one movie according to the dataset. So taking actors with atleast 2 movies and continue with the data.

In [26]:
query = f"""
WITH cte as (
  SELECT
    ROW_NUMBER() OVER (PARTITION BY actor_year.PID ORDER BY PID) row,
    PID, real_year
  FROM (
    SELECT DISTINCT TRIM(mc.PID) PID, {year_formula} real_year
    FROM Movie m JOIN M_Cast mc ON m.MID = mc.MID
    WHERE PID <> 'None'
    ORDER BY PID, real_year
  ) actor_year
)
SELECT * FROM cte
"""
pd.read_sql_query(query, conn)

,row,PID,real_year
0,1,nm0000002,1959
1,1,nm0000027,1984
2,1,nm0000039,1953
3,1,nm0000042,1953
4,1,nm0000047,1970
5,1,nm0000073,1939
6,1,nm0000076,1977
7,1,nm0000092,2004
8,1,nm0000093,1997
9,1,nm0000096,2016


Same Code from above but now taking actors who acted in 2 movies or more.

In [27]:
query = f"""
WITH cte as (
  SELECT
    ROW_NUMBER() OVER (PARTITION BY actor_year.pid_t ORDER BY pid_t) row,
    pid_t, real_year
  FROM (
    SELECT DISTINCT TRIM(mc.PID) pid_t, {year_formula} real_year
    FROM Movie m JOIN M_Cast mc ON m.MID = mc.MID
    WHERE pid_t IN (
      SELECT pid_t FROM (
        SELECT TRIM(PID) pid_t, COUNT(DISTINCT MID) count FROM M_Cast
        GROUP BY pid_t
      ) WHERE count > 1
    )
    ORDER BY pid_t, real_year
  ) actor_year
)
SELECT * FROM cte
"""
pd.read_sql_query(query, conn)

,row,pid_t,real_year
0,1,nm0000096,2016
1,2,nm0000096,2017
2,1,nm0000193,1996
3,2,nm0000193,2018
4,1,nm0000246,1996
5,2,nm0000246,2013
6,1,nm0000375,2005
7,2,nm0000375,2012
8,3,nm0000375,2016
9,1,nm0000673,2002


There is no definite range given in the original question so I am assuming the starting year of a actor is his/her career start and the last year he/she acted is his/her end of career.

If we consider like this and see the actor with id 'nm0000096' acted only in 2016 and 2017. So he is not unemployed more than 3 years considering his career started and ended at that years.

and if we consider actor with id 'nm9969854' he acted only in 1986, 2003 and 2008. So as he/she has gaps more than 3 years. So I dont consider this actor in my answer. (Both actor id's took are from start and end of the above table result).

By this assumption we may not get good results as lot of actors have only 2 movies which are not far apart year wise, and all those will be considered in our answer.

<p>
    
    Now let us get the difference between the rows of same pid's
</p>

In [28]:
query = f"""
WITH cte as (
  SELECT
    ROW_NUMBER() OVER (PARTITION BY actor_year.pid_t ORDER BY pid_t) row,
    pid_t PID, real_year year
  FROM (
    SELECT DISTINCT TRIM(mc.PID) pid_t, {year_formula} real_year
    FROM Movie m JOIN M_Cast mc ON m.MID = mc.MID
    WHERE pid_t IN (
      SELECT pid_t FROM (
        SELECT TRIM(PID) pid_t, COUNT(DISTINCT MID) count FROM M_Cast
        GROUP BY pid_t
      ) WHERE count > 1
    )
    ORDER BY pid_t, real_year
  ) actor_year
)


SELECT a.PID, b.year - a.year
FROM cte a JOIN cte b ON a.PID = b.PID AND (a.row = b.row - 1)
"""
pd.read_sql_query(query, conn)

,PID,b.year - a.year
0,nm0000096,1
1,nm0000193,22
2,nm0000246,17
3,nm0000375,7
4,nm0000375,4
5,nm0000673,14
6,nm0000818,1
7,nm0000818,1
8,nm0000818,1
9,nm0000818,1


As the above table seems correct we can continue to the **FINAL QUERY**

In [29]:
query = f"""
WITH cte as (
  SELECT
    ROW_NUMBER() OVER (PARTITION BY actor_year.pid_t ORDER BY pid_t) row,
    pid_t PID, real_year year
  FROM (
    SELECT DISTINCT TRIM(mc.PID) pid_t, {year_formula} real_year
    FROM Movie m JOIN M_Cast mc ON m.MID = mc.MID
    WHERE pid_t IN (
      SELECT pid_t FROM (
        SELECT TRIM(PID) pid_t, COUNT(DISTINCT MID) count FROM M_Cast
        GROUP BY pid_t
      ) WHERE count > 1
    )
    ORDER BY pid_t, real_year
  ) actor_year
)

SELECT DISTINCT TRIM(Name) FROM Person
WHERE TRIM(PID) IN (
  SELECT PID FROM (
    SELECT a.PID, (b.year - a.year) year_diff
    FROM cte a JOIN cte b ON a.PID = b.PID AND (a.row = b.row - 1)
  ) GROUP BY PID
  HAVING MAX(year_diff) <= 3
)
"""
pd.read_sql_query(query, conn)

,TRIM(Name)
0,Steven Hauck
1,Serena Williams
2,Raj Awasti
3,Michael Chapman
4,James Heron
5,Alex Jaep
6,James Pimenta
7,Elena Valdameri
8,M'laah Kaur Singh
9,Stephen Bullard


We got 3090 actors who are not unemployed for more than 3 years.

We got this high value because of the assumption I made, which is mentioned above. To get a satisfied answer there must be a range of years in which we should consider the careers of the actors. I assumed the career started with the starting year and ended with the last year. So the actors with only 2 movies nearby are all included in the answer. And I removed actors who only acted in one year so that our answer will less.

### Question 9:

Getting count of movies for every actor and director. So the following output table have actor id and director id as unique to every row.

In [30]:
query = """
SELECT actor, director, no_of_movies FROM (
  SELECT TRIM(mc.PID) actor, TRIM(md.PID) director, COUNT(DISTINCT md.MID) no_of_movies
  FROM M_Director md JOIN M_Cast mc ON md.MID = mc.MID
  GROUP BY actor, director
)
ORDER BY no_of_movies DESC
"""
pd.read_sql_query(query, conn)

,actor,director,no_of_movies
0,nm0456094,nm0223522,23
1,nm0007106,nm0223522,20
2,nm0434318,nm0223522,20
3,nm0318622,nm0080315,19
4,nm0332871,nm0223522,17
5,nm0712546,nm0698184,16
6,nm2147526,nm0698184,15
7,nm0442479,nm0223522,14
8,nm0451272,nm0080315,14
9,nm0451600,nm0223522,14


Getting a table with actor as key and the director with he/she had done maximum number of movies. If the director id is equal to yash chopra, then we should take that actor in our answer but we also have to check if there is any other director who have same number of movies with same actor. So according to question the actors who have same maximum number of movies for more than one director shouldnt be allowed in the answer.

In [31]:
query = """
WITH count_of_movies AS (
  SELECT actor, director, no_of_movies FROM (
    SELECT TRIM(mc.PID) actor, TRIM(md.PID) director, COUNT(DISTINCT md.MID) no_of_movies
    FROM M_Director md JOIN M_Cast mc ON md.MID = mc.MID
    GROUP BY actor, director
  )
), max_count_movies AS (
  SELECT actor, director, no_of_movies FROM count_of_movies
  WHERE (actor, no_of_movies) IN (
    SELECT actor, MAX(no_of_movies) no_of_movies FROM count_of_movies
    GROUP BY actor
  )
)

SELECT actor, director, no_of_movies FROM max_count_movies
WHERE actor IN (
  SELECT actor FROM max_count_movies
  GROUP BY actor
  HAVING COUNT(director) = 1
)
ORDER BY no_of_movies DESC
"""
pd.read_sql_query(query, conn)

,actor,director,no_of_movies
0,nm0456094,nm0223522,23
1,nm0007106,nm0223522,20
2,nm0434318,nm0223522,20
3,nm0318622,nm0080315,19
4,nm0332871,nm0223522,17
5,nm0712546,nm0698184,16
6,nm2147526,nm0698184,15
7,nm0442479,nm0223522,14
8,nm0451272,nm0080315,14
9,nm0451600,nm0223522,14


If the director id in above table is "Yash Chopra's" then the corresponding actor should be in answer.

Now joining the table with M_Cast and M_Director to get our **FINAL ANSWER**.

In [34]:
query = """
WITH count_of_movies AS (
  SELECT actor, director, no_of_movies FROM (
    SELECT TRIM(mc.PID) actor, TRIM(md.PID) director, COUNT(DISTINCT md.MID) no_of_movies
    FROM M_Director md JOIN M_Cast mc ON md.MID = mc.MID
    GROUP BY actor, director
  )
), max_count_movies AS (
  SELECT actor, director, no_of_movies FROM count_of_movies
  WHERE (actor, no_of_movies) IN (
    SELECT actor, MAX(no_of_movies) no_of_movies FROM count_of_movies
    GROUP BY actor
  )
)

SELECT DISTINCT TRIM(Name) FROM Person
WHERE TRIM(PID) IN (
  SELECT actor FROM (
    SELECT actor, director, no_of_movies FROM max_count_movies
    WHERE actor IN (
      SELECT actor FROM max_count_movies
      GROUP BY actor
      HAVING COUNT(director) = 1
    )
  ) WHERE director IN (
    SELECT TRIM(PID) FROM Person
    WHERE Name = 'Yash Chopra'
  )
)
"""
pd.read_sql_query(query, conn)

,TRIM(Name)
0,Waheeda Rehman
1,Achala Sachdev
2,Yash Chopra
3,Vinod Negi
4,Chandni Jas Keerat
5,Shivaya Singh
6,Huzefa Gadiwala
7,Manish Arora
8,Pankaj Raina
9,Neetu Singh


So we got 105 actors who made more movies with yash chopra than any other directors.

### Question 10:

First calculating Shahrukh 1 actors.

In [35]:
query = """
SELECT DISTINCT TRIM(PID) FROM M_Cast
WHERE MID IN (
  SELECT MID FROM M_Cast
  WHERE TRIM(PID) IN (
    SELECT TRIM(PID) FROM Person WHERE TRIM(Name) = 'Shah Rukh Khan'
  )
)
"""
pd.read_sql_query(query, conn)

,TRIM(PID)
0,nm0451321
1,nm0004418
2,nm1995953
3,nm2778261
4,nm0631373
5,nm0241935
6,nm0792116
7,nm1300111
8,nm0196375
9,nm1464837


So there are 2383 actors who acted with Shah Rukh Khan (Wow! :P). These are ShahRukh 1 actors. Now we have to calculate actors who are not in this list and are not 'Shah Rukh Khan' and acted with ShahRukh 1 actors.
This next query is going to be nested a lot and going to be **FINAL QUERY**.

In [36]:
query = """
SELECT DISTINCT TRIM(Name) FROM Person
WHERE TRIM(PID) IN (
  SELECT DISTINCT TRIM(PID) FROM M_Cast
  WHERE MID IN (
    SELECT MID FROM M_Cast
    WHERE TRIM(PID) IN (
      SELECT DISTINCT TRIM(PID) FROM M_Cast
      WHERE MID IN (
        SELECT MID FROM M_Cast
        WHERE TRIM(PID) IN (
          SELECT TRIM(PID) FROM Person WHERE TRIM(Name) = 'Shah Rukh Khan'
        )
      )
    )
  ) AND TRIM(PID) NOT IN (
    SELECT DISTINCT TRIM(PID) FROM M_Cast
    WHERE MID IN (
      SELECT MID FROM M_Cast
      WHERE TRIM(PID) IN (
        SELECT TRIM(PID) FROM Person WHERE TRIM(Name) = 'Shah Rukh Khan'
      )
    )
  ) AND TRIM(PID) NOT IN (
    SELECT TRIM(PID) FROM Person WHERE TRIM(Name) = 'Shah Rukh Khan'
  )
)
"""
pd.read_sql_query(query, conn)

,TRIM(Name)
0,Freida Pinto
1,Rohan Chand
2,Damian Young
3,Waris Ahluwalia
4,Caroline Christl Long
5,Rajeev Pahuja
6,Michelle Santiago
7,Alicia Vikander
8,Dominic West
9,Walton Goggins


There are 24308 actors who are ShahRukh 2 actors. And the ShahRukh 1 actors are not included in this. And Shah Rukh Khan is also not included in this list.

***Default Template for query***

In [ ]:
query = """

"""
pd.read_sql_query(query, conn)